In [9]:
!pip install gradio pdfplumber pandas python-dateutil openai matplotlib
!pip install pytesseract pillow pdf2image
!apt install tesseract-ocr -y
!pip install gradio openai pdfplumber pdf2image pytesseract python-dateutil matplotlib nest_asyncio
!pip install openai==0.28

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


In [ ]:
"""
💰 UPI Analyzer (PhonePe + Google Pay) using Gradio + OpenAI GPT
-----------------------------------------------------------------
- Works with both text-based and scanned PDFs.
- Auto-detects whether the statement is from PhonePe or GPay.
- Extracts, categorizes, visualizes, and summarizes transactions.
"""

import io, re, os, warnings
import pdfplumber
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import openai
from dateutil import parser as dateparser
from pdf2image import convert_from_bytes
import pytesseract
from PIL import Image
warnings.filterwarnings("ignore")

# ---------------- PDF Extraction ----------------
def extract_text_from_pdf(file_bytes: bytes) -> str:
    """Extract text from text-based or scanned PDFs (with OCR fallback)."""
    text = ""
    try:
        with pdfplumber.open(io.BytesIO(file_bytes)) as pdf:
            for page in pdf.pages:
                t = page.extract_text()
                if t:
                    text += "\n" + t
        if text.strip():
            print("[INFO] Text extracted via pdfplumber.")
            return text
    except Exception as e:
        print("[WARN] pdfplumber failed:", e)
    try:
        images = convert_from_bytes(file_bytes)
        for img in images:
            t = pytesseract.image_to_string(img, lang="eng")
            text += "\n" + t
        print("[INFO] Text extracted via OCR.")
    except Exception as e:
        print("[ERROR] OCR failed:", e)
    return text.strip()

# ---------------- Text Cleaner ----------------
def clean_text(raw: str) -> str:
    """Normalize Unicode, fix ₹ spacing, and remove noise."""
    text = re.sub(r"[�•]+", " ", raw)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"(?<=₹)\s+", "", text)
    return text

# ---------------- Parser Detection ----------------
def detect_source(text: str) -> str:
    """Heuristically detect whether PDF is PhonePe or GPay."""
    if "PhonePe" in text or "DEBIT" in text or "CREDIT" in text:
        return "phonepe"
    if "UPI Transaction ID" in text or "Paid to" in text:
        return "gpay"
    return "unknown"

# ---------------- PhonePe Parser ----------------
def parse_phonepe_text(text: str) -> pd.DataFrame:
    text = clean_text(text)
    pattern = re.compile(
        r"(?P<date>[A-Za-z]{3,9}\s+\d{1,2},?\s+\d{4}).{0,40}?"
        r"(?P<type>DEBIT|CREDIT)\s*[₹Rs\.\s]*(?P<amount>[\d,]+(?:\.\d{1,2})?)"
        r"(?P<desc>.+?)(?=(?:[A-Za-z]{3,9}\s+\d{1,2},?\s+\d{4})|Page|$)",
        re.DOTALL
    )
    entries = []
    for m in pattern.finditer(text):
        d = m.groupdict()
        try:
            date_obj = dateparser.parse(d["date"], fuzzy=True).date()
        except:
            continue
        desc = " ".join(d["desc"].split())
        entries.append({
            "date": date_obj,
            "type": d["type"].lower(),
            "amount": float(d["amount"].replace(",", "")),
            "description": desc
        })
    df = pd.DataFrame(entries)
    return finalize_df(df)

# ---------------- GPay Parser ----------------
def parse_gpay_text(text: str) -> pd.DataFrame:
    """Parse Google Pay statements."""
    text = clean_text(text)
    pattern = re.compile(
        r"(Paid to|Received from)\s+(?P<party>[A-Za-z0-9\s&]+).*?"
        r"(?:₹|Rs\.?)\s*(?P<amount>[\d,]+(?:\.\d{1,2})?)",
        re.DOTALL
    )
    entries = []
    for m in pattern.finditer(text):
        t = m.group(1)
        entry_type = "debit" if "Paid" in t else "credit"
        entries.append({
            "date": None,  # GPay statements often omit dates
            "type": entry_type,
            "amount": float(m.group("amount").replace(",", "")),
            "description": m.group("party").strip()
        })
    df = pd.DataFrame(entries)
    return finalize_df(df)

# ---------------- Common Helpers ----------------
def finalize_df(df):
    if df.empty:
        print("[WARN] No transactions parsed.")
        return df
    df["merchant"] = df["description"].apply(guess_merchant)
    df["category"] = df["description"].apply(categorize)
    return df

def guess_merchant(desc: str) -> str:
    m = re.search(r"(Paid to|Transfer to|Received from)\s+([A-Za-z\s&]+)", desc)
    return m.group(2).strip() if m else desc[:40]

def categorize(desc: str) -> str:
    desc = desc.lower()
    if "rent" in desc: return "rent"
    if "recharge" in desc or "mobile" in desc: return "utilities"
    if "loan" in desc: return "finance"
    if "amazon" in desc or "zepto" in desc or "shopping" in desc: return "shopping"
    if any(w in desc for w in ["food", "restaurant", "hotel", "swiggy", "zomato"]): return "food"
    if any(w in desc for w in ["google", "netflix", "subscription"]): return "entertainment"
    if "razorpay" in desc or "transfer" in desc: return "transfer"
    return "other"

# ---------------- Analysis ----------------
def analyze(df: pd.DataFrame) -> dict:
    result = {}
    if df.empty:
        return result
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    total_spent = df[df["type"] == "debit"]["amount"].sum()
    total_received = df[df["type"] == "credit"]["amount"].sum()
    result["summary"] = {
        "total_spent": total_spent,
        "total_received": total_received,
        "net": total_received - total_spent,
    }
    if df["date"].notnull().any():
        df["month"] = df["date"].dt.to_period("M")
        monthly = df.groupby(["month", "type"])["amount"].sum().unstack(fill_value=0)
    else:
        monthly = pd.DataFrame()
    result["monthly"] = monthly
    result["categories"] = (
        df[df["type"] == "debit"]
        .groupby("category")["amount"]
        .sum()
        .sort_values(ascending=False)
    )
    result["df"] = df
    return result

# ---------------- LLM + Visualization ----------------
def prompt_llm(analysis: dict) -> str:
    s = analysis["summary"]
    text = f"""
You are a financial advisor. Analyze this user's UPI statement.

Total spent: ₹{s['total_spent']:.2f}
Total received: ₹{s['total_received']:.2f}
Net flow: ₹{s['net']:.2f}

Top spending categories:
{analysis['categories'].to_string()}

Suggest 5 personalized financial improvement tips and a monthly budgeting plan.
"""
    return text

def call_openai(prompt: str, api_key: str, model="gpt-5"):
    openai.api_key = api_key or os.getenv("OPENAI_API_KEY")
    try:
        resp = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.6,
            max_tokens=400,
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"OpenAI API error: {e}"

def plot_monthly(monthly):
    fig, ax = plt.subplots(figsize=(6, 3))
    if not monthly.empty:
        monthly.plot(kind="bar", ax=ax)
    ax.set_title("Monthly Debit/Credit Summary")
    ax.set_ylabel("Amount (₹)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    return fig

def plot_categories(cat):
    fig, ax = plt.subplots(figsize=(5, 3))
    if not cat.empty:
        cat.plot(kind="barh", ax=ax)
    ax.set_title("Spending by Category")
    ax.set_xlabel("Amount (₹)")
    plt.tight_layout()
    return fig

# ---------------- Pipeline ----------------
def process_upi(pdf_files, api_key, model_choice):
    try:
        all_text = ""
        for f in pdf_files:
            path = f.name if hasattr(f, "name") else f
            print(f"[INFO] Reading {path}")
            with open(path, "rb") as file:
                data = file.read()
                all_text += "\n" + extract_text_from_pdf(data)

        src = detect_source(all_text)
        print(f"[INFO] Detected source: {src}")

        if src == "phonepe":
            df = parse_phonepe_text(all_text)
        elif src == "gpay":
            df = parse_gpay_text(all_text)
        else:
            return "❌ Unknown or unsupported PDF format.", None, None, "", ""

        if df.empty:
            return "❌ No valid transactions found. Ensure it’s a readable PhonePe or GPay statement.", None, None, "", ""

        analysis = analyze(df)
        prompt = prompt_llm(analysis)
        advice = call_openai(prompt, api_key, model_choice)

        fig1 = plot_monthly(analysis["monthly"])
        fig2 = plot_categories(analysis["categories"])

        html_table = df.head(200).to_html(index=False)
        summary = f"""
✅ Total Spent: ₹{analysis['summary']['total_spent']:.2f}
💰 Total Received: ₹{analysis['summary']['total_received']:.2f}
📊 Net Flow: ₹{analysis['summary']['net']:.2f}
"""
        return html_table, fig1, fig2, advice, summary
    except Exception as e:
        import traceback
        return f"Error: {e}\n{traceback.format_exc()}", None, None, "", ""

# ---------------- Gradio UI ----------------
def build_ui():
    with gr.Blocks(title="UPI Analyzer (PhonePe + GPay)") as demo:
        gr.Markdown("# 💰 UPI Analyzer (PhonePe + Google Pay) — Gradio + OpenAI")
        with gr.Row():
            with gr.Column(scale=2):
                pdf_input = gr.File(label="Upload UPI Statement PDF", file_count="multiple", type="filepath")
                model_choice = gr.Dropdown(["gpt-5-mini","gpt-5"], value="gpt-4-turbo", label="OpenAI Model")
                api_key = gr.Textbox(label="OpenAI API Key", placeholder="sk-...", type="password")
                analyze_btn = gr.Button("🔍 Analyze")
            with gr.Column(scale=3):
                summary = gr.Markdown("")
                table = gr.HTML("")
                advice_box = gr.Textbox(label="AI Recommendations", lines=10)
        with gr.Row():
            plot1 = gr.Plot(label="Monthly Summary")
            plot2 = gr.Plot(label="Category Breakdown")
        analyze_btn.click(
            fn=process_upi,
            inputs=[pdf_input, api_key, model_choice],
            outputs=[table, plot1, plot2, advice_box, summary],
        )
    return demo

# ---------------- Launch ----------------
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    demo = build_ui()
    demo.launch(share=True, debug=True,inline=False)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://731b5e0349f1ffab66.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

[INFO] Reading /tmp/gradio/6b7dc38b70195cabdc93fcbcb8ece2e20790e3ec8f6a5b32120047c22a47fa85/Paytm_UPI_Statement_31_Oct24_-_30_Oct25.pdf
[INFO] Text extracted via pdfplumber.
[INFO] Detected source: gpay


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

[INFO] Reading /tmp/gradio/5a8e3006bc0fe6e24bf6e8beb2d415085725580b6e1a3e7b5f85c0ba3f93c1e4/PhonePe_Statement_Oct2024_Oct2025.pdf


[INFO] Text extracted via pdfplumber.
[INFO] Detected source: phonepe
